In [265]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import openpyxl
from sklearn.pipeline import Pipeline
from datetime import datetime, timedelta
from sklearn.base import BaseEstimator, TransformerMixin

# **Generate Fake Data**

In [343]:
def random_date(start, end):
    return start + timedelta(days=random.randint(0, int((end - start).days)))


np.random.seed(42)
num_records = 10000

loan_ids = np.arange(1, num_records + 1)
disbursement_dates = [
    random_date(datetime(2020, 1, 1), datetime(2023, 1, 1)) for _ in range(num_records)
]
expire_dates = [d + timedelta(days=random.randint(30, 365)) for d in disbursement_dates]
is_employed = np.random.choice([True, False, np.nan], num_records, p=[0.7, 0.25, 0.05])
loan_amounts = np.random.choice(
    [np.nan, *np.random.uniform(-1000, 50000, num_records)], num_records
)
number_of_defaults = np.random.choice(
    [np.nan, *np.random.randint(0, 10, num_records)], num_records
)
outstanding_balances = np.random.choice(
    [np.nan, *np.random.uniform(0, 30000, num_records)], num_records
)
interest_rates = np.random.choice(
    [np.nan, *np.random.uniform(0.01, 0.25, num_records)], num_records
)
ages = np.random.choice([np.nan, *np.random.randint(18, 65, num_records)], num_records)
remaining_terms = np.random.choice(
    [np.nan, *np.random.randint(1, 60, num_records)], num_records
)
salaries = np.random.choice(
    [np.nan, *np.random.uniform(-10000, 20000, num_records)], num_records
)
loan_statuses = np.random.choice(
    ["Default", "Non-Default", np.nan], num_records, p=[0.3, 0.65, 0.05]
)
sectors = np.random.choice(
    ["Agriculture", "Manufacturing", "Services", "IT", "Retail"], num_records
)
currencies = np.random.choice(["USD", "EUR", "ZWL", "GBP", "AUD"], num_records)
employee_sectors = np.random.choice(
    ["Public", "Private", "Self-employed", "Unemployed"], num_records
)
statuses = np.random.choice(["Active", "Inactive"], num_records)

loan_data = pd.DataFrame(
    {
        "loan_id": loan_ids,
        "disbursement_date": disbursement_dates,
        "expire_date": expire_dates,
        "is_employed": is_employed,
        "loan_amount": loan_amounts,
        "number_of_defaults": number_of_defaults,
        "outstanding_balance": outstanding_balances,
        "interest_rate": interest_rates,
        "age": ages,
        "remaining_term": remaining_terms,
        "salary": salaries,
        "sector": sectors,
        "currency": currencies,
        # "employee_sector": employee_sectors,
        # "status": statuses,
        "loan_status": loan_statuses,
    }
)

##### Create Duplicates
num_duplicates = 100
duplicates = loan_data.sample(n=num_duplicates, replace=True).reset_index(drop=True)
loan_data = pd.concat([loan_data, duplicates], ignore_index=True)

In [344]:
loan_data.head(3)

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
0,1,2021-12-26,2022-03-30,1.0,28982.170288,4.0,29869.899084,0.112340,38.0,49.0,4908.585021,Agriculture,USD,Non-Default
1,2,2022-01-01,2022-09-25,NaN,27464.842065,5.0,27408.000182,0.128941,30.0,30.0,2283.627180,Agriculture,EUR,Non-Default
2,3,2021-02-03,2021-10-04,0.0,24469.175558,2.0,19078.850400,0.224119,63.0,17.0,4093.206305,Services,USD,Non-Default


In [345]:
loan_data.shape

(10100, 14)

# Data Cleanign steps

- Data cleaning is the process of ensuring that data is in the proper format, *making it suitable for analysis and modeling*

### 1.Check missing values 

In [346]:
loan_data.isnull().sum()

loan_id                  0
disbursement_date        0
expire_date              0
is_employed            478
loan_amount              1
number_of_defaults       1
outstanding_balance      2
interest_rate            1
age                      1
remaining_term           3
salary                   0
sector                   0
currency                 0
loan_status              0
dtype: int64

In [347]:
category_columns = loan_data.select_dtypes("number").columns
category_columns

Index(['loan_id', 'is_employed', 'loan_amount', 'number_of_defaults',
       'outstanding_balance', 'interest_rate', 'age', 'remaining_term',
       'salary'],
      dtype='object')

In [348]:
category_columns = loan_data.select_dtypes("object").columns
category_columns

Index(['sector', 'currency', 'loan_status'], dtype='object')

In [349]:
data = loan_data.copy()

In [350]:
duplicates = data.loc[data.duplicated(keep=False)].sort_values("loan_id")
duplicates

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
312,313,2020-03-25,2021-03-12,1.0,20887.773416,9.0,4965.231331,0.126386,37.0,32.0,5918.435398,Manufacturing,ZWL,Non-Default
10018,313,2020-03-25,2021-03-12,1.0,20887.773416,9.0,4965.231331,0.126386,37.0,32.0,5918.435398,Manufacturing,ZWL,Non-Default
579,580,2022-10-10,2023-05-10,1.0,35175.516492,8.0,28213.374035,0.241272,52.0,50.0,-6876.003554,Services,ZWL,Non-Default
10081,580,2022-10-10,2023-05-10,1.0,35175.516492,8.0,28213.374035,0.241272,52.0,50.0,-6876.003554,Services,ZWL,Non-Default
10034,595,2020-07-16,2020-12-01,0.0,31323.209397,3.0,1511.835714,0.014322,52.0,22.0,7798.739957,Agriculture,ZWL,Non-Default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10007,9798,2022-12-15,2023-11-03,1.0,10684.947309,6.0,26704.261346,0.121897,27.0,1.0,17557.191193,Manufacturing,EUR,nan
9946,9947,2022-05-31,2023-01-30,1.0,12806.860668,2.0,22788.492854,0.213088,33.0,12.0,-9136.527167,Retail,AUD,Default
10088,9947,2022-05-31,2023-01-30,1.0,12806.860668,2.0,22788.492854,0.213088,33.0,12.0,-9136.527167,Retail,AUD,Default
10003,9949,2020-09-06,2021-02-10,1.0,35573.093762,8.0,11054.830129,0.139636,28.0,33.0,13652.848693,Manufacturing,USD,Non-Default


In [351]:
def duplicates_check(df: pd.DataFrame):
    return data.loc[data.duplicated(keep=False)].sort_values("loan_id")

In [352]:
num_columns = [
    "loan_amount",
    "number_of_defaults",
    "outstanding_balance",
    "interest_rate",
    "age",
    "remaining_term",
    "salary",
]

In [353]:
data.head()

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
0,1,2021-12-26,2022-03-30,1.0,28982.170288,4.0,29869.899084,0.112340,38.0,49.0,4908.585021,Agriculture,USD,Non-Default
1,2,2022-01-01,2022-09-25,NaN,27464.842065,5.0,27408.000182,0.128941,30.0,30.0,2283.627180,Agriculture,EUR,Non-Default
2,3,2021-02-03,2021-10-04,0.0,24469.175558,2.0,19078.850400,0.224119,63.0,17.0,4093.206305,Services,USD,Non-Default
3,4,2022-06-07,2023-02-22,1.0,36218.404504,9.0,5269.500606,0.125747,30.0,7.0,13755.258629,Retail,GBP,Default
4,5,2021-09-18,2022-01-02,1.0,45353.259460,6.0,22912.296526,0.142642,40.0,10.0,14089.467579,Retail,EUR,Default


In [354]:
def check_missing_values(df: pd.DataFrame):
    return data.loc[data.isnull().any(axis=1)]

In [355]:
class CheckMissingValues(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.errors = pd.DataFrame()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        missing_counts = X.isnull().sum()
        missing_counts = missing_counts[missing_counts > 0]
        self.errors = pd.DataFrame(
            {"Column": missing_counts.index, "Missing Values": missing_counts.values}
        )

        return self.errors

In [356]:
missing = CheckMissingValues()
frms = missing.fit_transform(data)
frms

,Column,Missing Values
0,is_employed,478
1,loan_amount,1
2,number_of_defaults,1
3,outstanding_balance,2
4,interest_rate,1
5,age,1
6,remaining_term,3


In [357]:
check_missing_values = data.loc[data.isnull().any(axis=1)]
check_missing_values

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
1,2,2022-01-01,2022-09-25,NaN,27464.842065,5.0,27408.000182,0.128941,30.0,30.0,2283.627180,Agriculture,EUR,Non-Default
11,12,2021-07-11,2021-11-10,NaN,9899.361717,1.0,4842.795849,0.192387,27.0,38.0,-2135.138884,IT,EUR,Non-Default
34,35,2021-09-12,2022-03-12,NaN,10926.266823,3.0,14616.721703,0.189646,24.0,24.0,2609.446486,Manufacturing,EUR,Non-Default
50,51,2020-03-21,2020-09-07,NaN,34120.093785,0.0,9980.403155,0.243796,39.0,2.0,18455.388926,Agriculture,AUD,Default
69,70,2022-01-19,2023-01-03,NaN,23488.845535,7.0,15711.060298,0.209259,27.0,14.0,-6018.926070,IT,USD,Non-Default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10005,6267,2020-10-30,2021-06-27,NaN,43439.224403,1.0,13526.688453,0.042392,34.0,41.0,-3852.851532,Manufacturing,ZWL,Default
10029,4678,2020-01-13,2021-01-06,NaN,33955.737775,9.0,10839.297775,0.073503,56.0,59.0,-8005.482590,IT,ZWL,Non-Default
10058,1617,2022-04-06,2022-08-15,NaN,3942.330169,7.0,22910.580293,0.134743,31.0,12.0,-3302.427098,IT,ZWL,Default
10071,2496,2022-09-05,2023-08-19,1.0,17358.037769,1.0,6242.926831,0.194841,51.0,NaN,8746.591592,Agriculture,GBP,Default


In [358]:
def check_invalid_dates(df: pd.DataFrame):
    dates_invalid = df[df["disbursement_date"] > df["expire_date"]]
    return dates_invalid

In [359]:
dates_invalid = data[data["disbursement_date"] > data["expire_date"]]
dates_invalid

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status


In [360]:
mandatory_columns = [
    "loan_id",
    "disbursement_date",
    "expire_date",
    "is_employed",
    "loan_amount",
    "number_of_defaults",
    "outstanding_balance",
    "interest_rate",
    "age",
    "remaining_term",
    "salary",
    "sector",
    "currency",
    "employee_sector",
    "status",
    "loan_status",
]

In [361]:
def check_mandatory_columns(df: pd.DataFrame, mandatory_columns: list) -> list:
    missing_columns = [col for col in mandatory_columns if col not in df.columns]
    if missing_columns:
        return missing_columns
    else:
        return None

In [362]:
class MandatoryColumns(BaseEstimator, TransformerMixin):
    def __init__(self, mandatory_columns):
        self.mandatory_columns = mandatory_columns
        self.errors = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        missing_columns = [
            col for col in self.mandatory_columns if col not in X.columns
        ]
        if missing_columns:
            self.errors = missing_columns
        else:
            self.errors = []
        return X

    def get_errors(self):
        return self.errors

In [363]:
num_columns_ck = ["loan_amount", "interest_rate", "age", "salary"]

In [364]:
conditions = [(data[col] < 0) | (data[col] == 0) for col in num_columns_ck]
mask = pd.concat(conditions, axis=1).any(axis=1)
check_negative_amounts_and_zeros = data[mask]
check_negative_amounts_and_zeros

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
5,6,2022-01-21,2022-11-24,1.0,5974.506204,0.0,2492.023710,0.014792,63.0,47.0,-1529.950804,IT,ZWL,Non-Default
6,7,2021-04-17,2021-08-11,1.0,21934.173342,0.0,9747.824979,0.143750,22.0,36.0,-8570.523443,IT,ZWL,Non-Default
9,10,2021-05-07,2021-12-26,0.0,41763.628792,0.0,5985.145076,0.110431,21.0,31.0,-2405.424979,Agriculture,USD,Non-Default
10,11,2020-08-10,2021-07-27,1.0,31032.257305,8.0,8698.407086,0.098271,22.0,29.0,-2592.699823,Retail,AUD,Non-Default
11,12,2021-07-11,2021-11-10,NaN,9899.361717,1.0,4842.795849,0.192387,27.0,38.0,-2135.138884,IT,EUR,Non-Default
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10089,4604,2021-07-23,2021-09-25,1.0,8089.888044,9.0,16659.070767,0.227382,54.0,34.0,-9477.399211,IT,ZWL,Non-Default
10093,4904,2020-07-03,2020-11-07,1.0,19832.856833,6.0,3211.339731,0.045099,55.0,30.0,-8205.592246,Services,ZWL,Default
10094,712,2022-09-09,2022-12-07,1.0,10216.151950,1.0,26682.023733,0.096406,43.0,26.0,-908.600442,Retail,ZWL,Default
10096,4788,2020-05-31,2021-05-23,0.0,32633.463593,9.0,16284.579850,0.069643,59.0,32.0,-8419.546504,IT,EUR,Default


In [365]:
def check_negative_amounts_and_zeros(df: pd.DataFrame):
    conditions = [(data[col] < 0) | (data[col] == 0) for col in num_columns_ck]
    mask = pd.concat(conditions, axis=1).any(axis=1)
    negative_amounts_and_zeros = data[mask]
    negative_amounts_and_zeros
    return negative_amounts_and_zeros

In [366]:
data[num_columns] = data[num_columns].apply(pd.to_numeric, errors="coerce")
not_converted_num = data.loc[data[num_columns].isnull().any(axis=1)]

In [367]:
def converted_num(df: pd.DataFrame):
    num_columns = [
        "loan_amount",
        "number_of_defaults",
        "outstanding_balance",
        "interest_rate",
        "age",
        "remaining_term",
        "salary",
    ]
    data[num_columns] = data[num_columns].apply(pd.to_numeric, errors="coerce")
    not_converted_num = data.loc[data[num_columns].isnull().any(axis=1)]
    return not_converted_num

In [368]:
not_converted_num

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status
427,428,2020-10-16,2020-12-16,0.0,49449.532816,NaN,18552.233004,0.043065,35.0,25.0,19121.373969,IT,USD,Non-Default
1159,1160,2021-02-26,2021-09-15,1.0,30245.723605,3.0,4089.793095,NaN,30.0,28.0,-6032.421389,Agriculture,AUD,Non-Default
2495,2496,2022-09-05,2023-08-19,1.0,17358.037769,1.0,6242.926831,0.194841,51.0,NaN,8746.591592,Agriculture,GBP,Default
2498,2499,2022-10-12,2023-08-02,NaN,48750.015415,6.0,NaN,0.028080,27.0,6.0,7509.425339,Retail,AUD,Default
2837,2838,2020-07-11,2021-04-23,1.0,23774.989832,4.0,25581.870683,0.167046,31.0,NaN,-9841.307132,Retail,AUD,Non-Default
3241,3242,2021-01-10,2021-02-18,1.0,10968.860177,4.0,22488.176453,0.213830,NaN,11.0,6929.077248,Retail,AUD,Non-Default
5483,5484,2022-03-15,2023-01-30,0.0,47403.951535,2.0,NaN,0.040812,44.0,7.0,11909.320534,Manufacturing,EUR,Non-Default
7381,7382,2022-05-18,2023-02-22,1.0,NaN,6.0,16856.131988,0.226779,32.0,55.0,19476.052291,Retail,EUR,Default
10071,2496,2022-09-05,2023-08-19,1.0,17358.037769,1.0,6242.926831,0.194841,51.0,NaN,8746.591592,Agriculture,GBP,Default


In [369]:
dates_columns = loan_data.filter(regex="date").columns
category_columns = loan_data.select_dtypes("object").columns

In [370]:
for column in dates_columns:
    data[column] = pd.to_datetime(data[column], format="%d/%m/%Y", errors="coerce")

In [371]:
not_converted_dates = data.loc[data[dates_columns].isnull().any(axis=1)]
not_converted_dates

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status


In [372]:
def converted_dates(df: pd.DataFrame):
    dates_columns = df.filter(regex="date").columns
    for column in dates_columns:
        df[column] = pd.to_datetime(df[column], format="%d/%m/%Y", errors="coerce")
        not_converted_dates = data.loc[data[dates_columns].isnull().any(axis=1)]
        return not_converted_dates

In [373]:
class DateConverter(BaseEstimator, TransformerMixin):
    def __init__(self, date_formats=None):
        self.errors = None
        self.date_formats = (
            date_formats
            if date_formats is not None
            else ["%d/%m/%Y", "%Y-%m-%d", "%m/%d/%Y", "%d-%m-%Y", "%Y.%m.%d"]
        )

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        dates_columns = X.filter(regex="date").columns
        X_temp = X.copy()
        not_converted_dates = pd.DataFrame()

        for date_format in self.date_formats:
            try:
                X_temp[dates_columns] = X_temp[dates_columns].apply(
                    pd.to_datetime, format=date_format, errors="coerce"
                )
            except Exception:
                continue
            if X_temp[dates_columns].isnull().any().any():
                not_converted_dates = X_temp[X_temp[dates_columns].isnull().any(axis=1)]
                if not not_converted_dates.empty:
                    break
            else:
                break

        return not_converted_dates

In [374]:
class ConvertedNumeric(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X):
        return self

    def transform(self, X):
        X = data.copy()
        num_columns = [
            "loan_amount",
            "number_of_defaults",
            "outstanding_balance",
            "interest_rate",
            "age",
            "remaining_term",
            "salary",
        ]
        X[num_columns] = X[num_columns].apply(pd.to_numeric, errors="coerce")
        not_converted_num = X.loc[X[num_columns].isnull().any(axis=1)]
        return not_converted_num

In [375]:
class CheckNegativeAmountsAndZerosAmounts(BaseEstimator, TransformerMixin):
    def __init__(self, num_columns_ck=None):
        if num_columns_ck is None:
            num_columns_ck = ["loan_amount", "interest_rate", "age", "salary"]
        self.num_columns_ck = num_columns_ck
        self.errors = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise ValueError
        missing_cols = [col for col in self.num_columns_ck if col not in X.columns]
        if missing_cols:
            raise ValueError(f"Missing columns: {', '.join(missing_cols)}")
        conditions = [(X[col] < 0) | (X[col] == 0) for col in self.num_columns_ck]
        mask = pd.concat(conditions, axis=1).any(axis=1)
        negative_amounts_and_zeros = X[mask]
        return negative_amounts_and_zeros

In [376]:
class CheckDuplicates(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.errors = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            raise ValueError
        duplicates = X.loc[X.duplicated(keep=False)].sort_values("loan_id")
        return duplicates

In [377]:
class CheckInvalidDates(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.errors = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        dates_columns = X[X["disbursement_date"] > X["expire_date"]]
        self.errors = dates_columns
        return dates_columns

In [378]:
pipeline = Pipeline(
    [
        ("mandatory_columns", MandatoryColumns(mandatory_columns=mandatory_columns)),
        ("check_missing_values", CheckMissingValues()),
        ("date_converter", DateConverter()),
        # ('check_invalid_dates', CheckInvalidDates()),
        ("convert_numeric", ConvertedNumeric()),
        ("check_negative_amounts_and_zeros", CheckNegativeAmountsAndZerosAmounts()),
        ("check_duplicates", CheckDuplicates()),
    ]
)

In [379]:
df_cleaned = data.copy()
pipeline.fit(df_cleaned)
pipeline.transform(df_cleaned)

,loan_id,disbursement_date,expire_date,is_employed,loan_amount,number_of_defaults,outstanding_balance,interest_rate,age,remaining_term,salary,sector,currency,loan_status


In [380]:
with pd.ExcelWriter("data_issues.xlsx") as writer:
    mandatory_errors = pipeline.named_steps["mandatory_columns"].get_errors()
    if mandatory_errors is not None:
        pd.DataFrame(mandatory_errors, columns=["Missing Mandatory Columns"]).to_excel(
            writer, sheet_name="Missing Mandatory Columns"
        )
    else:
        pd.DataFrame(columns=["Missing Mandatory Columns"]).to_excel(
            writer, sheet_name="Missing Mandatory Columns", index=False
        )

    missing_values = pipeline.named_steps["check_missing_values"].errors
    if missing_values is not None and not missing_values.empty:
        missing_values.to_excel(writer, sheet_name="Missing Values")
    else:
        pd.DataFrame(columns=["Column", "Missing Values"]).to_excel(
            writer, sheet_name="Missing Values", index=False
        )

    invalid_dates = pipeline.named_steps["date_converter"].errors
    if invalid_dates is not None and not invalid_dates.empty:
        invalid_dates.to_excel(writer, sheet_name="Invalid Date Conversion")
    else:
        pd.DataFrame(columns=["Invalid Dates"]).to_excel(
            writer, sheet_name="Invalid Date Conversion", index=False
        )

    numeric_conversion_issues = pipeline.named_steps["convert_numeric"].transform(
        df_cleaned
    )
    if numeric_conversion_issues is not None and not numeric_conversion_issues.empty:
        numeric_conversion_issues.to_excel(
            writer, sheet_name="Numeric Conversion Issues"
        )
    else:
        pd.DataFrame(columns=["Numeric Conversion Issues"]).to_excel(
            writer, sheet_name="Numeric Conversion Issues", index=False
        )

    negative_amounts_and_zeros = pipeline.named_steps[
        "check_negative_amounts_and_zeros"
    ].transform(df_cleaned)
    if negative_amounts_and_zeros is not None and not negative_amounts_and_zeros.empty:
        negative_amounts_and_zeros.to_excel(
            writer, sheet_name="Negative Amounts and Zeros"
        )
    else:
        pd.DataFrame(columns=["Negative Amounts and Zeros"]).to_excel(
            writer, sheet_name="Negative Amounts and Zeros", index=False
        )

    duplicates = pipeline.named_steps["check_duplicates"].transform(df_cleaned)
    if duplicates is not None and not duplicates.empty:
        duplicates.to_excel(writer, sheet_name="Duplicates")
    else:
        pd.DataFrame(columns=["Duplicates"]).to_excel(
            writer, sheet_name="Duplicates", index=False
        )